In [67]:
import pandas as pd
import os

In [68]:
#get NER output 
#NOTE: mergedNER is just the seperate files concatenated together "cat * > ../mergedNER.tsv"
NERdf = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t")
NERdf.head(2)

,Unnamed: 0,Unnamed: 0.1,key,content,topics
0,0.0,0,andalusiastarnews_Remember_When:_Shehan’s_Poli...,Dan Shehan’s “My Polio Story” is continued in ...,"[('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '..."
1,1.0,1,andalusiastarnews_Veterans_Foundation_salutes_...,"This past Sunday, March 29th marked National V...","[('DATE', 'This past Sunday, March 29th'), ('E..."


In [15]:
newsDf = pd.read_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/localNational.tsv", sep="\t")
newsDf.head(2)

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (2,3,7,10,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,article_id,sourcedomain_id,date,title,content,url_x,fips,source,...,predrt_0,predrt_12,predrt_3,national,id,author,url,published,published_utc,collection_utc
0,0,0.0,andalusiastarnews--2020-04-04--Remember When: ...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Remember When: Shehan’s Polio story,Dan Shehan’s “My Polio Story” is continued in ...,https://www.andalusiastarnews.com/2020/04/03/r...,1039.0,andalusiastarnews,...,25.04,44.15,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,andalusiastarnews--2020-04-04--Veterans Founda...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Veterans Foundation salutes Vietnam veterans,"This past Sunday, March 29th marked National V...",https://www.andalusiastarnews.com/2020/04/03/v...,1039.0,andalusiastarnews,...,25.04,44.15,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
NERdf = NERdf.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'content'])
NERdf = NERdf.rename(columns={"url_x":"url"})
NERdf.head(3)

,key,topics
0,andalusiastarnews_Remember_When:_Shehan’s_Poli...,"[('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '..."
1,andalusiastarnews_Veterans_Foundation_salutes_...,"[('DATE', 'This past Sunday, March 29th'), ('E..."
2,andalusiastarnews_Governor_issues_stay_at_home...,"[('PERSON', 'Kay Ivey'), ('DATE', 'Saturday'),..."


In [81]:
#newsDf = newsDf.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

#make sure columns used to make the key are unique and don't have na values 
newsDf = newsDf.drop_duplicates(["source", "title"]).dropna(subset=["source", "title"])

#add key to newsDf so we can merge the NER data on it 
def underscore(inStr): 
    return inStr.replace(" ", "_")

newsDf["key"] = newsDf["source"] + "_" + newsDf["title"].apply(underscore) 
newsDf.head(2)

,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,onlinesince,...,predrt_12,predrt_3,national,id,author,url,published,published_utc,collection_utc,key
0,andalusiastarnews--2020-04-04--Remember When: ...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Remember When: Shehan’s Polio story,Dan Shehan’s “My Polio Story” is continued in ...,https://www.andalusiastarnews.com/2020/04/03/r...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",15-Oct-1998,...,44.15,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Remember_When:_Shehan’s_Poli...
1,andalusiastarnews--2020-04-04--Veterans Founda...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Veterans Foundation salutes Vietnam veterans,"This past Sunday, March 29th marked National V...",https://www.andalusiastarnews.com/2020/04/03/v...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",15-Oct-1998,...,44.15,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Veterans_Foundation_salutes_...


In [39]:
#newsDf["date"] = pd.to_datetime(newsDf["date"])
#half2020 = newsDf[(newsDf["date"] >= "2020-01-01") & (newsDf["date"] <= "2020-08-01")]
#earlyJan.shape

(114472, 43)

In [82]:
merged = pd.merge(newsDf, NERdf, how = "inner", on="key")

In [86]:
print(merged.shape)
merged.head(3)

(4769869, 45)


,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,onlinesince,...,predrt_3,national,id,author,url,published,published_utc,collection_utc,key,topics
0,andalusiastarnews--2020-04-04--Remember When: ...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Remember When: Shehan’s Polio story,Dan Shehan’s “My Polio Story” is continued in ...,https://www.andalusiastarnews.com/2020/04/03/r...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",15-Oct-1998,...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Remember_When:_Shehan’s_Poli...,"[('PERSON', 'Dan Shehan’s'), ('WORK_OF_ART', '..."
1,andalusiastarnews--2020-04-04--Veterans Founda...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Veterans Foundation salutes Vietnam veterans,"This past Sunday, March 29th marked National V...",https://www.andalusiastarnews.com/2020/04/03/v...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",15-Oct-1998,...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Veterans_Foundation_salutes_...,"[('DATE', 'This past Sunday, March 29th'), ('E..."
2,andalusiastarnews--2020-04-04--Governor issues...,andalusiastarnews-andalusiastarnews.com,2020-04-04,Governor issues stay at home order,Governor Kay Ivey announced that a stay at hom...,https://www.andalusiastarnews.com/2020/04/03/g...,1039.0,andalusiastarnews,"Serves Covington County, located in Andalusia....",15-Oct-1998,...,30.8,False,NaN,NaN,NaN,NaN,NaN,NaN,andalusiastarnews_Governor_issues_stay_at_home...,"[('PERSON', 'Kay Ivey'), ('DATE', 'Saturday'),..."


In [84]:
#looks like we were able to merge with every row successfully 
newsDf.shape

(4769869, 44)

In [91]:
import csv
merged.to_csv("/shared/3/projects/benlitterer/localNews/mergedNewsData/mergedNER.tsv", sep="\t", quoting=csv.QUOTE_NONNUMERIC)

In [89]:
merged.loc[1,"topics"]

"[('DATE', 'This past Sunday, March 29th'), ('EVENT', 'National Vietnam War Veterans Day'), ('ORDINAL', 'first'), ('DATE', 'March 29, 2012'), ('PERSON', 'Obama'), ('DATE', 'annual'), ('ORG', 'Congress'), ('PERSON', 'Trump'), ('DATE', '2017'), ('DATE', 'March 29'), ('GPE', 'U.S.'), ('GPE', 'Vietnam'), ('DATE', 'March 29, 1973'), ('EVENT', 'the Paris Peace Accords'), ('CARDINAL', '2.5 million'), ('NORP', 'American'), ('CARDINAL', 'millions'), ('DATE', 'the period 1961-1975'), ('ORG', 'The Covington Veterans Foundation'), ('ORG', 'the Vietnam Veterans'), ('ORG', 'U.S. Army'), ('ORG', '2nd Battalion'), ('ORG', '2nd Infantry Regiment'), ('ORG', '1st Infantry Division'), ('DATE', 'August 11, 1966')]"

In [ ]:
#create a with entities as keys and documents as values 
